### 필요 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import os
import librosa
import pickle
import tensorflow as tf
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import tensorflow.keras.layers as layers
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 압축해제

In [ ]:
!unzip '/content/drive/MyDrive/USG2_data.zip' -d '/content/drive/MyDrive/[라떼들]문제2번/'

### 사용자 경로 지정

In [ ]:
path = "/content/drive/MyDrive/[라떼들]문제2번/" #공통된 디렉토리

In [ ]:
train_data = pd.read_csv(path+'train_label.csv')
submit = pd.read_csv(path+'answer_sample.csv')
train_data.head()

,file,label
0,train_0001.npy,0
1,train_0002.npy,0
2,train_0003.npy,0
3,train_0004.npy,0
4,train_0005.npy,0


In [ ]:
Y_train = train_data['label']
Y_train

0       0
1       0
2       0
3       0
4       0
       ..
3541    0
3542    0
3543    0
3544    0
3545    1
Name: label, Length: 3546, dtype: int64

### origin_fre_train 파일 만들기

In [ ]:
def extract_fft_features(directory):
    global left_f
    features=[]
    sr = 16000
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            data = np.load(f)
            fft = np.fft.fft(data)
            magnitude = np.abs(fft)
            freq = np.linspace(0,16000,len(magnitude))
            left_spectrum = magnitude[:int(len(magnitude)/2)]
            left_f = freq[:int(len(magnitude)/2)]
            features.append(left_spectrum)
    return features

In [ ]:
features_train=extract_fft_features(path+"train_data/")

In [ ]:
train_df = pd.DataFrame(features_train, columns=left_f)
train_df

,0.000000,0.200003,0.400005,0.600008,0.800010,1.000013,1.200015,1.400018,1.600020,1.800023,...,7998.099976,7998.299979,7998.499981,7998.699984,7998.899986,7999.099989,7999.299991,7999.499994,7999.699996,7999.899999
0,3686.0,3725.684347,4230.607624,5338.879215,3247.527295,7429.496789,3853.989817,20107.318894,4674.084629,8336.863776,...,371.522190,1414.998741,678.575599,2610.788074,1541.645223,384.812389,295.823095,4294.155354,3094.157039,1204.543653
1,3882.0,2469.576468,1989.978855,2471.123083,5410.616286,3432.694737,7831.611429,6531.505058,5266.520255,6365.515560,...,3596.322075,5184.918616,826.496791,5141.130226,4586.474298,1921.275990,7388.852258,1389.979828,2048.800932,1530.704952
2,9129.0,7790.532053,8717.473106,6840.361275,10437.555418,5533.442321,4661.188873,9069.707849,10739.005753,12446.800079,...,1723.827804,3425.178332,2323.528540,3867.739324,8317.205533,2965.203206,4822.973998,3026.108805,5777.272315,3647.988757
3,1384.0,1593.544255,1270.284128,1828.503502,1504.045999,1518.212177,1410.742994,2031.071137,1591.308317,1646.127374,...,7505.784198,5455.455686,16458.056293,3662.890208,12640.630423,6907.412275,13380.876872,16978.954075,4440.252465,12311.007869
4,12262.0,13639.344102,12157.783573,10494.062403,16380.983207,13052.570525,53659.725293,22089.558726,25325.088262,45706.676900,...,7541.460822,17936.067223,35242.286094,13319.326349,22162.807842,15164.672384,23503.249294,17632.578783,23838.423278,28941.482090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,1118.0,6893.129904,2171.511234,4485.408638,6319.620134,20339.107620,18415.686198,19644.104814,29358.856392,19353.516511,...,6172.159634,3327.214119,16261.747220,3399.347317,11385.289716,8696.291797,14999.429256,13888.041256,3487.818212,11657.857255
3542,11385.0,1106.549012,764.310106,1041.612596,2720.249845,2664.210890,5131.188861,8286.655269,5279.805478,2558.214067,...,6524.378422,3776.493744,4588.260433,4526.668836,5478.119826,4776.334214,2482.901967,5241.546474,4110.475714,7543.828155
3543,3037.0,653.137936,1719.648678,1904.182680,2987.742338,3206.778312,4600.194559,3250.291500,10793.472465,14423.995060,...,3106.861817,4287.146144,973.439162,15048.674137,4710.830238,4840.907035,2948.001484,7897.421623,5589.153850,3706.060178
3544,2348.0,1476.917964,3635.114442,4282.246942,4433.397885,11888.202987,20929.322563,29707.209338,58045.039201,32068.010780,...,10020.204170,18000.986886,5931.649931,7460.182103,3293.607390,15219.530545,9934.135501,2551.415079,17785.277442,8826.899567


In [ ]:
print(path)

/content/drive/MyDrive/[라떼들]문제2번/


### train_mfcc 파일 생성

In [ ]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # MFCC 벡터를 추출할 개수
    'SEED':41
}

In [ ]:
from tqdm import tqdm
def get_mfcc_feature(home_directory,directory):
    features =[]
    df = pd.read_csv(home_directory+'train_label.csv')  ### 여기 경로를 지정해줘야함. (train_label 있는 곳으로..)
    for filename in tqdm(os.listdir(directory)):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            data = np.load(f)#,sr=CFG['SR'])
            mfcc = librosa.feature.mfcc(y=data, sr=16000, n_mfcc=CFG['N_MFCC'])
            y_feature = []
            for e in (mfcc):
                y_feature.append(np.mean(e))
            features.append(y_feature)

    # # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    df = pd.concat([df, mfcc_df], axis=1)

    print('Done.')
    return df

In [ ]:
train_mfcc = get_mfcc_feature(path,path+"train_data")

100%|██████████| 3546/3546 [01:46<00:00, 33.28it/s]

Done.


In [ ]:
train_mfcc.head(4)

,file,label,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,train_0001.npy,0,709.806396,101.648117,-4.307143,20.930332,-8.851012,4.311610,-6.523489,-2.094957,...,-1.451989,1.788547,-0.184718,-1.404901,1.245032,4.481526,5.206417,5.412833,2.790755,5.030035
1,train_0002.npy,0,707.063721,84.921799,-1.154310,16.953890,0.470908,6.941554,1.918724,2.855398,...,0.124267,2.096994,1.909655,1.424736,0.486897,-0.640957,-2.243032,-0.087248,-2.734498,-0.538931
2,train_0003.npy,0,699.085571,85.460114,-0.956208,16.642735,3.089166,7.232333,1.839488,3.138356,...,0.617879,1.648731,1.283655,0.357120,-0.959492,-0.956217,-2.133114,0.917144,0.649397,1.700940
3,train_0004.npy,0,721.375916,40.294914,6.358726,11.532204,1.109248,-4.310456,4.602643,8.218973,...,-9.481628,5.118536,5.123003,-4.977022,-5.000679,-3.745972,-8.824956,-1.926467,10.098228,5.868020


### 데이터 전처리

In [ ]:
# 데이터 프레임 범위 자를거면 이렇게
train_df = train_df.loc[:,:3000]

In [ ]:
train_mfcc

,file,label,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,mfcc_23,mfcc_24,mfcc_25,mfcc_26,mfcc_27,mfcc_28,mfcc_29,mfcc_30,mfcc_31,mfcc_32
0,train_0001.npy,0,709.806396,101.648117,-4.307143,20.930332,-8.851012,4.311610,-6.523489,-2.094957,...,-1.451989,1.788547,-0.184718,-1.404901,1.245032,4.481526,5.206417,5.412833,2.790755,5.030035
1,train_0002.npy,0,707.063721,84.921799,-1.154310,16.953890,0.470908,6.941554,1.918724,2.855398,...,0.124267,2.096994,1.909655,1.424736,0.486897,-0.640957,-2.243032,-0.087248,-2.734498,-0.538931
2,train_0003.npy,0,699.085571,85.460114,-0.956208,16.642735,3.089166,7.232333,1.839488,3.138356,...,0.617879,1.648731,1.283655,0.357120,-0.959492,-0.956217,-2.133114,0.917144,0.649397,1.700940
3,train_0004.npy,0,721.375916,40.294914,6.358726,11.532204,1.109248,-4.310456,4.602643,8.218973,...,-9.481628,5.118536,5.123003,-4.977022,-5.000679,-3.745972,-8.824956,-1.926467,10.098228,5.868020
4,train_0005.npy,0,801.232910,26.237694,-8.623388,4.819234,2.049866,16.483395,8.277191,1.130842,...,5.017206,-0.661709,-0.289784,-0.367740,4.580405,0.343925,-1.743407,0.126307,-1.217743,0.243455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541,train_3542.npy,0,732.008972,48.548405,3.614643,10.330624,4.330559,12.110374,14.497869,4.511088,...,-2.236179,-2.607800,-7.327556,1.210373,8.167914,2.593621,-5.004297,-1.679092,5.088736,3.641206
3542,train_3543.npy,0,661.790527,92.014603,14.850841,-3.871539,8.949002,6.359140,6.906483,2.500548,...,-3.976672,-2.882119,-0.795929,-6.688501,-9.397826,1.565515,13.246170,5.768762,-1.255437,-7.830335
3543,train_3544.npy,0,687.458923,88.954926,18.170395,0.695201,4.542387,4.582627,2.514268,0.569916,...,-7.533755,-7.328268,-2.787889,-5.069835,4.336580,13.898629,8.964062,-3.341093,-5.199396,-5.458975
3544,train_3545.npy,0,746.171448,53.930470,-14.329453,8.228229,8.790443,13.696578,2.979938,9.128135,...,2.950100,1.327892,-2.612118,1.819103,3.282863,-2.699971,-2.918215,-0.469597,4.018785,3.440809


In [ ]:
feature = ['mfcc_2', 'mfcc_14', 'mfcc_17', 'mfcc_30', 'mfcc_12', 'mfcc_7',
       'mfcc_29', 'mfcc_22', 'mfcc_27', 'mfcc_9']
train_mfcc = train_mfcc[feature]

In [ ]:
train_tot = pd.concat([train_df,train_mfcc],axis=1)

test_tot.to_csv(path+'data/test_tot.csv',index=False)

### test scale 적용을 위한 train scaler 파일 저장

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler  # 표준화 패키지 라이브러리

x = train_tot.values
y = train_data['label'].values

scaler = MinMaxScaler()
scaler = scaler.fit(x)

with open(path + 'data/scaler.pkl','wb') as f:
  pickle.dump(scaler,f)

x = scaler.transform(x)

features = [train_tot.columns]
standard_scale_train = pd.DataFrame(x, columns=features)
standard_scale_train.head(5)

,0.0,0.2000025000312504,0.4000050000625008,0.6000075000937511,0.8000100001250016,1.0000125001562519,1.2000150001875023,1.4000175002187527,1.6000200002500031,1.8000225002812535,...,mfcc_2,mfcc_14,mfcc_17,mfcc_30,mfcc_12,mfcc_7,mfcc_29,mfcc_22,mfcc_27,mfcc_9
0,0.040918,0.040316,0.046887,0.052307,0.040754,0.086116,0.040034,0.250002,0.043748,0.044386,...,0.729198,0.314785,0.720416,0.741793,0.481990,0.199386,0.663480,0.426416,0.618504,0.340891
1,0.043095,0.026653,0.021842,0.024120,0.068831,0.039207,0.082775,0.080928,0.049366,0.033745,...,0.623480,0.341283,0.548188,0.541804,0.462757,0.434947,0.438204,0.715285,0.591079,0.614745
2,0.101374,0.084528,0.097038,0.067065,0.134081,0.063863,0.048707,0.112539,0.101257,0.066570,...,0.626883,0.366922,0.530232,0.578325,0.525545,0.432736,0.441528,0.662108,0.538757,0.579366
3,0.015350,0.017125,0.013798,0.017804,0.018123,0.016738,0.013780,0.024879,0.014517,0.008270,...,0.341418,0.434793,0.178544,0.474929,0.810909,0.509835,0.239162,0.630551,0.392572,0.338366
4,0.136172,0.148144,0.135492,0.102977,0.211228,0.152112,0.575224,0.274689,0.239564,0.246102,...,0.252570,0.616475,0.303940,0.549569,0.565900,0.612365,0.453313,0.696369,0.739158,0.464016


In [ ]:
from sklearn.model_selection import train_test_split

# train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(standard_scale_train, Y_train, test_size=0.2, stratify=Y_train, random_state=22)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

rf = RandomForestClassifier(n_estimators=100,random_state=100)
gb = XGBClassifier(n_estimators=100,random_state=100)
MLP = MLPClassifier()
estimators = [('RF', rf),
          ('GB', gb),
          ('mlp', MLP)]

Model = StackingClassifier(estimators=estimators, final_estimator=MLPClassifier())

Model.fit(x_train,y_train).score(x_valid,y_valid)

/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.8183098591549296

### 모델 저장

In [ ]:
import joblib
# model save
saved_model = joblib.dump(Model,path+'model/model.pkl')